In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 2.9 MB/s eta 0:00:0000:0100:01m


In [2]:
import cv2
import numpy as np
import mediapipe as mp

# Configuración de MediaPipe
mp_hands = mp.solutions.hands.Hands(
    static_image_mode=False, 
    max_num_hands=1, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
cap = cv2.VideoCapture(0)  # Iniciar la cámara web

# Almacenamiento de datos y etiquetas
data, labels = [], []

def capture_gesture(label):
    """Captura gestos y guarda los landmarks junto con la etiqueta."""
    print(f"Grabando gesto {label}. Presiona 'q' para detener.")
    
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Refleja la imagen horizontalmente
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
                data.append(landmarks)  # Guardar landmarks
                labels.append(label)    # Guardar etiqueta del gesto

                # Dibujar los puntos en la mano detectada
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS
                )

        cv2.imshow('Grabación de Gestos', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Captura de gestos con etiquetas 0: Piedra, 1: Papel, 2: Tijeras
for i, gesture in enumerate(["Piedra (0)", "Papel (1)", "Tijeras (2)"]):
    input(f"Presiona Enter para grabar el gesto: {gesture}")
    capture_gesture(i)

# Guardar los datos y etiquetas en archivos .npy
cap.release()
cv2.destroyAllWindows()
np.save('rps_dataset.npy', np.array(data))
np.save('rps_labels.npy', np.array(labels))
print("Dataset guardado con éxito en 'rps_dataset.npy' y 'rps_labels.npy'.")


I0000 00:00:1729366253.762061  247417 gl_context.cc:357] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1729366253.779455  253657 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729366253.793188  253657 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Grabando gesto 0. Presiona 'q' para detener.


/Users/pepeargentoo/AAII-2/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Grabando gesto 1. Presiona 'q' para detener.
Grabando gesto 2. Presiona 'q' para detener.
Dataset guardado con éxito en 'rps_dataset.npy' y 'rps_labels.npy'.


# train-gesture-classifier

In [4]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Cargar los datos
X = np.load('rps_dataset.npy')
y = np.load('rps_labels.npy')

# División del dataset en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo con la capa de entrada explícita
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 clases: piedra, papel, tijeras
])

# Compilar el modelo
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# Guardar el modelo en formato Keras
model.save('rps_model.keras')
print("Modelo entrenado y guardado como 'rps_model.keras'")


Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4624 - loss: 1.0000 - val_accuracy: 0.5877 - val_loss: 0.8859
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6992 - loss: 0.8162 - val_accuracy: 0.7325 - val_loss: 0.6615
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7555 - loss: 0.6176 - val_accuracy: 0.7632 - val_loss: 0.5176
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8108 - loss: 0.4641 - val_accuracy: 0.8947 - val_loss: 0.4198
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8851 - loss: 0.3712 - val_accuracy: 0.9211 - val_loss: 0.3152
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9290 - loss: 0.2950 - val_accuracy: 0.9386 - val_loss: 0.2323
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9375 - loss: 0.2283 - val_accuracy: 0.9605 - val_loss: 0.1796
Epoch 8/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9562 - loss: 0.1586 - val_accuracy: 0.9825 - val_loss:

# rock-paper-scissors.

In [5]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Cargar el modelo entrenado
model = tf.keras.models.load_model('rps_model.keras')

# Inicializar MediaPipe para la detección de manos
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Nombres de las clases
class_names = ['Piedra', 'Papel', 'Tijeras']

# Captura desde la cámara web
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Refleja la imagen para una interacción natural
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mp_hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
            
            # Realizar la predicción
            prediction = model.predict(landmarks[np.newaxis, ...])
            class_id = np.argmax(prediction)
            class_name = class_names[class_id]

            # Dibujar los puntos clave y el nombre de la clase
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            cv2.putText(frame, class_name, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar la imagen con las predicciones
    cv2.imshow('Piedra, Papel o Tijeras', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1729366706.692745  247417 gl_context.cc:357] GL version: 2.1 (2.1 ATI-6.1.13), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1729366706.706919  261539 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729366706.719324  261539 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


/Users/pepeargentoo/AAII-2/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━